In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

learning_rate = 1e-4
keep_prob_rate = 0.7 # 
max_epoch = 2000
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # 每一维度  滑动步长全部是 1， padding 方式 选择 same
    # 提示 使用函数  tf.nn.conv2d
    strides=[1, 1, 1, 1]
    return tf.nn.conv2d(x, W, strides, padding='SAME')

def max_pool_2x2(x):
    # 滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式 请选择 same
    # 提示 使用函数  tf.nn.max_pool
    ksize=[1, 2, 2, 1]
    strides=[1, 2, 2, 1]
    return tf.nn.max_pool(x, ksize, strides, padding='SAME')  

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 784])/255.
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1])

#  卷积层 1
## conv1 layer ##

W_conv1 = weight_variable([7, 7, 1, 32])                  # patch 7x7, in size 1, out size 32
b_conv1 = bias_variable([32])   
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  # 卷积层 + ReLU 激活函数  
h_pool1 = max_pool_2x2(h_conv1)                     # 池化               

# 卷积层 2
W_conv2 = weight_variable([5, 5, 32, 64])                 # patch 5x5, in size 32, out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  # 卷积层 + ReLU 激活函数
h_pool2 = max_pool_2x2(h_conv2)                      # 池化

#  全连接层 1
## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 全连接层 2
## fc2 layer ##
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# 交叉熵函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(max_epoch):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob:keep_prob_rate})
        if i % 100 == 0:
            print(compute_accuracy(
                mnist.test.images[:1000], mnist.test.labels[:1000]))


**因为我的平台上无法运行TensorFlow 1.x环境的代码，所以将代码又写了TensorFlow 2.x版本的**
**以下是在TensorFlow 2.x版本下的运行结果**

In [2]:
import tensorflow as tf  
from tensorflow.keras.datasets import mnist  
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  
from tensorflow.keras.optimizers import Adam  
  
# 数据加载  
(x_train, y_train), (x_test, y_test) = mnist.load_data()  
  
# 数据预处理  
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255  
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255  
y_train = tf.keras.utils.to_categorical(y_train, 10)  
y_test = tf.keras.utils.to_categorical(y_test, 10)  
  
# 构建模型  
model = Sequential()  
model.add(Conv2D(32, kernel_size=(7, 7), activation='relu', input_shape=(28, 28, 1)))  
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Conv2D(64, (5, 5), activation='relu'))  
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Flatten())  # 将多维的输入一维化，常用在从卷积层到全连接层的过渡  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.3))  # dropout层，保留概率是0.7  
model.add(Dense(10, activation='softmax'))  # 输出层，10个类别  
  
# 编译模型  
model.compile(loss='categorical_crossentropy',  
              optimizer=Adam(learning_rate=1e-4),  
              metrics=['accuracy'])  
  
# 训练模型  
batch_size = 100  
epochs = 2000  
  
# 使用tf.data创建数据集  
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))  
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(batch_size)  
  
# 训练模型  
history = model.fit(train_dataset, epochs=epochs, validation_data=(x_test, y_test))  
  
# 计算测试集准确率  
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)  
print('\nTest accuracy:', test_acc)

Epoch 1/2000
600/600 [==============================] - 21s 34ms/step - loss: 0.5159 - accuracy: 0.8618 - val_loss: 0.1628 - val_accuracy: 0.9518
Epoch 2/2000
600/600 [==============================] - 22s 37ms/step - loss: 0.1495 - accuracy: 0.9566 - val_loss: 0.0944 - val_accuracy: 0.9735
Epoch 3/2000
600/600 [==============================] - 22s 37ms/step - loss: 0.1019 - accuracy: 0.9694 - val_loss: 0.0751 - val_accuracy: 0.9761
Epoch 4/2000
600/600 [==============================] - 24s 40ms/step - loss: 0.0801 - accuracy: 0.9753 - val_loss: 0.0558 - val_accuracy: 0.9831
Epoch 5/2000
600/600 [==============================] - 24s 40ms/step - loss: 0.0664 - accuracy: 0.9795 - val_loss: 0.0472 - val_accuracy: 0.9855
Epoch 6/2000
600/600 [==============================] - 24s 40ms/step - loss: 0.0562 - accuracy: 0.9830 - val_loss: 0.0440 - val_accuracy: 0.9862
Epoch 7/2000
600/600 [==============================] - 24s 40ms/step - loss: 0.0500 - accuracy: 0.9848 - val_loss: 0.0383 -

KeyboardInterrupt: 